In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import sys
sys.version

'3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]'

In [3]:
import collections
import csv
import os

In [4]:
!pip install bert-tensorflow

In [5]:
from bert import modeling
from bert import optimization
from bert import tokenization

In [6]:
pip install tensorflow==1.15.4

Note: you may need to restart the kernel to use updated packages.


In [7]:
#from ipynb.fs.full.tokenization import *
#from ipynb.fs.full.modeling import *
import tensorflow as tf

In [8]:
print(tf.__version__)

1.15.4


In [9]:
pip install numpy==1.18.5

Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy
numpy.version.version

'1.18.5'

###### Using Dictionary to initialize values

###### Flags

In [11]:
flags = tf.flags

FLAGS = flags.FLAGS

flags.DEFINE_string(
    "bert_config_file","bert_config.json",
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")


flags.DEFINE_string("vocab_file","vocab.txt","The vocabulary file that the BERT model was trained on.")


#Other parameters

flags.DEFINE_string(
    "init_checkpoint","bert_model.ckpt",
    "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

flags.DEFINE_integer(
    "max_seq_length",512,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_bool("do_train",True, "Whether to run training.")

flags.DEFINE_bool("do_eval",True, "Whether to run eval on the dev set.")

flags.DEFINE_bool(
    "do_predict",True,
    "Whether to run the model in inference mode on the test set.")

flags.DEFINE_integer("train_batch_size", 4, "Total batch size for training.")

flags.DEFINE_integer("eval_batch_size", 8, "Total batch size for eval.")

flags.DEFINE_integer("predict_batch_size", 8, "Total batch size for predict.")

flags.DEFINE_float("learning_rate", 2e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 3.0,
                   "Total number of training epochs to perform.")

flags.DEFINE_float(
    "warmup_proportion", 0.1,
    "Proportion of training to perform linear learning rate warmup for. "
    "E.g., 0.1 = 10% of training.")

flags.DEFINE_integer("save_checkpoints_steps", 1000,
                     "How often to save the model checkpoint.")

flags.DEFINE_integer("iterations_per_loop", 1000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")

tf.flags.DEFINE_string(
    "tpu_name", None,
    "The Cloud TPU to use for training. This should be either the name "
    "used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 "
    "url.")

tf.flags.DEFINE_string(
    "tpu_zone", None,
    "[Optional] GCE zone where the Cloud TPU is located in. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

tf.flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

tf.flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_integer(
    "num_tpu_cores", 8,
    "Only used if `use_tpu` is True. Total number of TPU cores to use.")

# task specific parameter( sentiment analysis)
flags.DEFINE_integer("num_classes", 40, "Total number of labels for sentiment analysis")
flags.DEFINE_integer("num_aspects", 20, "Total number of aspect")

flags.DEFINE_list("aspect_value_list", [0, 1], "Values that a aspect can have")
tf.app.flags.DEFINE_string('f', '', 'kernel')

###### Making Data Ready

In [12]:
import pandas as pd

In [13]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """

In [14]:
class InputFeatures(object):

    def __init__(self, input_ids, input_mask, segment_ids, label_id, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.is_real_example=is_real_example

In [15]:
def convert_single_example(ex_index, example, label_list, max_seq_length,tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_id=0,
            is_real_example=False)

    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    # print("label_map:",label_map,";length of label_map:",len(label_map))
   # example.label =  example.label.replace(' ', ',')
    label_id = None
    if "," in example.label:  # multiple label
        # get list of label
        label_id_list = []
        label_list = example.label.split(",")
        for label_ in label_list:
            try:
                label_id_list.append(label_map[label_])
            except:
                tf.logging.error(label_)
                tf.logging.error(label_map)
                exit(1)
        label_id = [0 for l in range(len(label_map))]
        for j, label_index in enumerate(label_id_list):
            label_id[label_index] = 1
    else:  # single label
        label_id = label_map[example.label]
    if ex_index < 5:
        tf.logging.info("*** Example ***")
        tf.logging.info("guid: %s" % (example.guid))
        tf.logging.info("tokens: %s" % " ".join(
            [tokenization.printable_text(x) for x in tokens]))
        tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        if "," in example.label: tf.logging.info("label: %s (id_list = %s)" % (str(example.label),
                                                                               str(
                                                                                   label_id_list)))  # if label_id is a list, try print multi-hot value: label_id_list
        tf.logging.info("label: %s (id = %s)" % (str(example.label), str(label_id)))  # %d

    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_id = label_id,
        is_real_example=True)
    return feature

In [16]:
def file_based_convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example, label_list,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)

        if isinstance(feature.label_id, list):
            label_ids = feature.label_id
        else:
            label_ids = [feature.label_id]
        features["label_ids"] = create_int_feature(label_ids)
        #label_ids.extend(pad_sequence(label_ids,FLAGS.num_classes))
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()

In [17]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""
    # task specific parameter
    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([FLAGS.num_classes], tf.int64),  # ADD TO A FIXED LENGTH
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)
       # label_ids = tf.cast(example['label_ids'], dtype=tf.int32)
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn

In [18]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length=FLAGS.max_seq_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [19]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)

        probabilities = tf.nn.sigmoid(logits)
        labels = tf.cast(labels, tf.float32)
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)

In [20]:
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
            is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
            is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                            init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold_fn=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids,probabilities, is_real_example):
                # print("###metric_fn.logits:",logits.shape) # (?,80)
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # print("###metric_fn.label_ids:",label_ids.shape,";predictions:",predictions.shape) # label_ids: (?,80);predictions:(?,)
                logits_split = tf.split(probabilities, FLAGS.num_aspects,
                                        axis=-1)  # a list. length is num_aspects
                label_ids_split = tf.split(label_ids, FLAGS.num_aspects,
                                           axis=-1)  # a list. length is num_aspects
                accuracy = tf.constant(0.0, dtype=tf.float64)

                for j, logits in enumerate(logits_split):  #
                    #  accuracy = tf.metrics.accuracy(label_ids, predictions)

                    predictions = tf.argmax(logits, axis=-1,
                                            output_type=tf.int32)  # should be [batch_size,]
                    label_id_ = tf.cast(tf.argmax(label_ids_split[j], axis=-1), dtype=tf.int32)
                    tf.logging.debug("label_ids_split[j] = %s; predictions = %s; label_id_ = %s" %
                                     (label_ids_split[j], predictions, label_id_))
                    current_accuracy, update_op_accuracy = tf.metrics.accuracy(
                                    labels=label_id_, predictions= predictions, weights=is_real_example)
                    accuracy += tf.cast(current_accuracy, dtype=tf.float64)
                #accuracy = accuracy / tf.constant(FLAGS.num_aspects, dtype=tf.float64)
                loss = tf.metrics.mean(per_example_loss)
                return {
                    "eval_accuracy": (accuracy, update_op_accuracy),
                    "eval_loss": loss,
                }

            eval_metrics = (metric_fn,
                            [per_example_loss, label_ids,probabilities, is_real_example])
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metrics=eval_metrics,
                scaffold_fn=scaffold_fn)
        else:
            logits_split = tf.split(probabilities, FLAGS.num_aspects,
                                        axis=-1)  # a list. length is num_aspects
            label_ids_split = tf.split(label_ids, FLAGS.num_aspects,
                                           axis=-1)  # a list. length is num_aspects
            accuracy = tf.constant(0.0, dtype=tf.float64)

            for j, logits in enumerate(logits_split):  
                predictions = tf.argmax(logits, axis=-1,
                                            output_type=tf.int32)  # should be [batch_size,]
                label_id_ = tf.cast(tf.argmax(label_ids_split[j], axis=-1), dtype=tf.int32)
                tf.logging.debug("label_ids_split[j] = %s; predictions = %s; label_id_ = %s" %
                                     (label_ids_split[j], predictions, label_id_))
                current_accuracy, update_op_accuracy = tf.metrics.accuracy(
                                    labels=label_id_, predictions= predictions, weights=is_real_example)
                accuracy += tf.cast(current_accuracy, dtype=tf.float64)
            
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities,
                            "label_ids":label_ids
                            },
                scaffold_fn=scaffold_fn)
        return output_spec

    return model_fn

In [21]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    all_label_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.segment_ids)
        all_label_ids.append(feature.label_id)

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        num_examples = len(features)
        d = tf.data.Dataset.from_tensor_slices({
            "input_ids":
                tf.constant(
                    all_input_ids, shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "segment_ids":
                tf.constant(
                    all_segment_ids,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "label_ids":
                tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
        })

        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
        return d
    return input_fn

In [22]:
def convert_examples_to_features(examples, label_list, max_seq_length=FLAGS.max_seq_length,
                                 tokenizer= tokenization.FullTokenizer(vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example, label_list,
                                         max_seq_length, tokenizer)

        features.append(feature)
    return features

### Main Function

In [23]:
tf.compat.v1.logging.set_verbosity(tf.logging.INFO)

In [24]:
tokenization.validate_case_matches_checkpoint(FLAGS.do_lower_case,
                                                  FLAGS.init_checkpoint)

In [25]:
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

In [26]:
tokenizer = tokenization.FullTokenizer(
        vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

In [27]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [28]:
output_dir = r"C:\Users\u1111995\Desktop\project 2\Output8"

In [29]:
tf.gfile.MakeDirs(output_dir)

In [30]:
#Codie_label_path = "C:/Users/u1111995/Desktop/project 2/Codie_Data/Data"
#Labels_df = pd.read_csv((Codie_label_path + "/MapFile.txt"), sep="\t")
#df = pd.DataFrame(Labels_df['Label'])
#Labels_df

In [31]:
 def get_labels():
        """See base class."""
        label_list = []
        # num_aspect=FLAGS.num_aspects
        aspect_value_list = FLAGS.aspect_value_list  # [-2,-1,0,1]
        for i in range(FLAGS.num_aspects):
            for value in aspect_value_list:
                label_list.append(str(i) + "_" + str(value))
        return label_list  # [ {'0_-2': 0, '0_-1': 1, '0_0': 2, '0_1': 3,....'19_-2': 76, '19_-1': 77, '19_0': 78, '19_1': 79}]

In [32]:
label_list = get_labels()
label_list

['0_0',
 '0_1',
 '1_0',
 '1_1',
 '2_0',
 '2_1',
 '3_0',
 '3_1',
 '4_0',
 '4_1',
 '5_0',
 '5_1',
 '6_0',
 '6_1',
 '7_0',
 '7_1',
 '8_0',
 '8_1',
 '9_0',
 '9_1',
 '10_0',
 '10_1',
 '11_0',
 '11_1',
 '12_0',
 '12_1',
 '13_0',
 '13_1',
 '14_0',
 '14_1',
 '15_0',
 '15_1',
 '16_0',
 '16_1',
 '17_0',
 '17_1',
 '18_0',
 '18_1',
 '19_0',
 '19_1']

In [33]:
tpu_cluster_resolver = None
if FLAGS.use_tpu and FLAGS.tpu_name:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
            FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)


In [34]:
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [35]:
run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        master=FLAGS.master,
        model_dir=output_dir,
        save_checkpoints_steps=FLAGS.save_checkpoints_steps,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=FLAGS.iterations_per_loop,
            num_shards=FLAGS.num_tpu_cores,
            per_host_input_for_training=is_per_host))

### TrainExamples

In [36]:
data_dir = r"C:\Users\u1111995\Desktop\project 2\Codie_Data\Data"

In [37]:
filename = 'codietrain16.tsv'

In [38]:
import pandas as pd
gk = pd.read_csv(os.path.join(data_dir, "codietrain16.tsv"),sep="\t")

In [39]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, label, text_b=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

In [40]:
def _create_examples(gk, set_type):
    examples = []
    for i in range(len(gk)):
      #  if i == 0:
      #      continue
        guid = "%s-%s" % (set_type, i)
        label = tokenization.convert_to_unicode(gk.label[i])
        text_a = tokenization.convert_to_unicode(gk.notes[i])
        examples.append(
                InputExample(guid=guid, text_a=text_a,label=label,text_b=None))
    return examples

In [41]:
def get_train_examples(data_dir):
    return _create_examples(gk, "train")

In [42]:
train_examples = None
num_train_steps = None
num_warmup_steps = None

In [43]:
train_examples = get_train_examples(data_dir)
train_examples[0].label

'0_0,1_0,2_0,3_0,4_0,5_0,6_0,7_0,8_0,9_0,10_0,11_0,12_0,13_1,14_0,15_0,16_0,17_0,18_0,19_0'

In [44]:
num_train_steps = int(len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

In [45]:
num_train_steps

225

In [46]:
num_warmup_steps

22

In [47]:
len(label_list)

40

In [48]:
model_fn = model_fn_builder(
        bert_config=bert_config,
        num_labels=len(label_list),
        init_checkpoint=FLAGS.init_checkpoint,
        learning_rate=FLAGS.learning_rate,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        use_tpu=FLAGS.use_tpu,
        use_one_hot_embeddings=False)

In [49]:
estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=FLAGS.use_tpu,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=FLAGS.train_batch_size,
        eval_batch_size=FLAGS.eval_batch_size,
        predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\u1111995\\Desktop\\project 2\\Output8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000149101BEE48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_s

In [50]:
if FLAGS.do_train:
        train_file = os.path.join(output_dir, "train.tf_record")
        file_based_convert_examples_to_features(
            train_examples,label_list=label_list, max_seq_length = FLAGS.max_seq_length, tokenizer=tokenizer,output_file=train_file)
        tf.logging.info("***** Running training *****")
        tf.logging.info("  Num examples = %d", len(train_examples))
        tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)
        tf.logging.info("  Num steps = %d", num_train_steps)


INFO:tensorflow:Writing example 0 of 300
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-0
INFO:tensorflow:tokens: [CLS] a 46 - year - old male , with no relevant medical history , non ##smo ##ker . she underwent fixed rehabilitation on a single implant ( it ##i ##® sl ##a ) . st ##ra ##uman ##n . wal ##der ##burg - switzerland ) placed in zone 3 . 6 . six months after cement ##ing the crown , the patient came to control with no associated symptoms . clinical examination revealed slight red ##ness of the mu ##cos ##a adjacent to the implant and a 6 mm deep per ##i - implant pocket with mild bleeding on probing . the role of joint showed premature contact with pro ##st ##hetic crown . pan ##ora ##mic radio ##graphy showed a radio ##lu ##cent area in the marginal bone at 3 . 6 . the pro ##st ##hetic crown was o ##cc ##lus ##ally carved . subsequently , a mu ##cope ##rit ##ic flap was raised from 3 . 5 to 3 . 7 , a marginal flap of fi ##bro ##us tissue occupying a per ##i - i

In [51]:
train_input_fn = file_based_input_fn_builder(
            input_file=train_file,
            seq_length=FLAGS.max_seq_length,
            is_training=True,
            drop_remainder=True)

In [52]:
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:training_loop marked as finished


### EvalExamples

In [53]:
data_dir = r"C:\Users\u1111995\Desktop\project 2\Codie_Data\Data"

In [54]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, label, text_b=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

In [55]:
def _create_examples(gk, set_type):
    examples = []
    for i in range(len(gk)):
      #  if i == 0:
      #      continue
        guid = "%s-%s" % (set_type, i)
        label = tokenization.convert_to_unicode(gk.label[i])
        text_a = tokenization.convert_to_unicode(gk.notes[i])
        examples.append(
                InputExample(guid=guid, text_a=text_a,label=label,text_b=None))
    return examples

In [56]:
dev = pd.read_csv(os.path.join(data_dir, "codievalidation16.tsv"),sep="\t")
dev

,Unnamed: 0,articleID,notes,label
0,129,S0212-16112005000400011-1,"The patient, a 55-year-old man without known a...","0_0,1_0,2_0,3_1,4_0,5_0,6_0,7_0,8_1,9_0,10_0,1..."
1,183,S0365-66912006000600011-1,A 13-year-old male patient is diagnosed with p...,"0_0,1_0,2_1,3_0,4_0,5_0,6_0,7_0,8_0,9_0,10_0,1..."
2,216,S0365-66912011000800006-1,An 83-year-old male with advanced Alzheimer's ...,"0_0,1_0,2_0,3_0,4_0,5_0,6_1,7_0,8_0,9_1,10_0,1..."
3,182,S0365-66912006000600010-1,A 16-year-old woman was referred for consultat...,"0_0,1_0,2_0,3_0,4_0,5_0,6_1,7_0,8_0,9_0,10_0,1..."
4,349,S1130-14732006000300007-1,A 28-year-old male who was admitted to the eme...,"0_0,1_0,2_0,3_1,4_0,5_1,6_1,7_1,8_1,9_0,10_1,1..."
...,...,...,...,...
95,141,S0212-16112012000400060-1,A 31-year-old woman with a history of multiple...,"0_0,1_0,2_0,3_1,4_1,5_0,6_1,7_1,8_0,9_1,10_0,1..."
96,72,S0210-48062007000700014-1,"A 46-year-old male, with no relevant past medi...","0_0,1_0,2_0,3_0,4_1,5_0,6_0,7_0,8_0,9_0,10_0,1..."
97,24,S0004-06142007000500011-1,An 11-year-old white girl with a history of pr...,"0_0,1_0,2_0,3_1,4_1,5_0,6_0,7_1,8_0,9_0,10_0,1..."
98,449,S1139-76322015000100012-3,A nine-month-old male presented with intense p...,"0_0,1_1,2_0,3_0,4_0,5_0,6_0,7_0,8_1,9_0,10_0,1..."


In [57]:
def get_dev_examples(data_dir):
    return _create_examples(dev, "dev")

In [58]:
if FLAGS.do_eval:
    eval_examples = get_dev_examples(data_dir)
    if FLAGS.use_tpu:
        while len(eval_examples) % FLAGS.eval_batch_size != 0:
            eval_examples.append(PaddingInputExample())
    eval_file = os.path.join(output_dir, "eval.tf_record")
    file_based_convert_examples_to_features(eval_examples, label_list=label_list,max_seq_length=FLAGS.max_seq_length, tokenizer=tokenizer, 
                                            output_file=eval_file)        

INFO:tensorflow:Writing example 0 of 100
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: dev-0
INFO:tensorflow:tokens: [CLS] the patient , a 55 - year - old man without known all ##er ##gies diagnosed with gas ##tric aden ##oca ##rc ##ino ##ma , was admitted in november 2001 to the digest ##ive service of son du ##ret ##a hospital where a sub ##to ##tal gas ##tre ##ct ##omy and reconstruction of the gas ##tro ##jer ##o ii were performed . five days after surgery the patient ' s clinical condition requires the prescription of artificial nutrition , starting central administration of t ##p ##n . the preparation administered is ka ##bi ##mi ##x ##® ( see detailed composition in table i ) , a diet of 2 , 55 ##3 ki ##lo ##cal ##ories supplemented in the a care service with alternating days with a multi ##vita ##min preparation ##® ( vi ##tp ##m ##v ) . 48 hours after parent ##eral nutrition was established , it was decided to change the type of diet , in order to administer a formula 

In [59]:
num_actual_eval_examples = len(eval_examples)
tf.logging.info("***** Running evaluation *****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",len(eval_examples), num_actual_eval_examples,
                        len(eval_examples) - num_actual_eval_examples)
tf.logging.info("  Batch size = %d", FLAGS.eval_batch_size)

INFO:tensorflow:***** Running evaluation *****
INFO:tensorflow:  Num examples = 100 (100 actual, 0 padding)
INFO:tensorflow:  Batch size = 8


In [60]:
eval_steps = None
if FLAGS.use_tpu:
    assert len(eval_examples) % FLAGS.eval_batch_size == 0
    eval_steps = int(len(eval_examples) // FLAGS.eval_batch_size)
eval_drop_remainder = True if FLAGS.use_tpu else False
eval_input_fn = file_based_input_fn_builder(input_file=eval_file,seq_length=FLAGS.max_seq_length,is_training=False,
            drop_remainder=eval_drop_remainder)

In [61]:
result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?, 40)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use tf.where in 2.0, which has the s

f

In [63]:
output_eval_file = os.path.join(output_dir, "test_results.txt")
with tf.io.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:***** Eval results *****
INFO:tensorflow:  eval_accuracy = 0.9800000190734863
INFO:tensorflow:  eval_loss = 0.46788707
INFO:tensorflow:  global_step = 225
INFO:tensorflow:  loss = 0.46499076


In [64]:
f

NameError: name 'f' is not defined

### Test

In [65]:
test = pd.read_csv(os.path.join(data_dir, "codietest16.tsv"),sep="\t")
test

,Unnamed: 0,articleID,notes,label
0,223,S0365-66912012000700003-1,A 32-year-old male with colonic neoplasia with...,"0_0,1_0,2_0,3_1,4_0,5_0,6_0,7_0,8_0,9_0,10_0,1..."
1,258,S1130-01082005001200011-1,A 58-year-old man presented to his reference h...,"0_0,1_0,2_0,3_1,4_0,5_0,6_1,7_1,8_0,9_0,10_0,1..."
2,341,S1130-05582017000300150-2,An 8-year-old female patient with increased le...,"0_0,1_0,2_0,3_0,4_0,5_1,6_0,7_0,8_0,9_0,10_0,1..."
3,207,S0365-66912008000700008-1,A 64-year-old male presented with an emergency...,"0_0,1_0,2_0,3_0,4_0,5_0,6_1,7_1,8_0,9_0,10_0,1..."
4,267,S1130-01082007001100009-1,A 42-year-old woman was studied in Gastroenter...,"0_0,1_0,2_0,3_1,4_0,5_0,6_0,7_0,8_1,9_0,10_0,1..."
...,...,...,...,...
95,8,S0004-06142006000200014-1,"A 36-year-old male, with no personal history o...","0_0,1_0,2_0,3_0,4_1,5_0,6_0,7_1,8_0,9_0,10_0,1..."
96,45,S0004-06142010000100015-1,A 42-year-old man presented with a 2-month his...,"0_0,1_0,2_0,3_0,4_1,5_0,6_0,7_0,8_0,9_0,10_0,1..."
97,117,S0211-69952013000500035-1,We report the case of a 64-year-old man with n...,"0_0,1_0,2_0,3_1,4_1,5_0,6_0,7_1,8_0,9_1,10_0,1..."
98,205,S0365-66912008000100007-2,A girl of 11 years and 10 months old.\nNormal ...,"0_0,1_0,2_1,3_0,4_0,5_0,6_1,7_0,8_0,9_0,10_0,1..."


In [66]:
def get_test_examples(data_dir):
    return _create_examples(test, "test")

In [67]:
if FLAGS.do_predict:
        predict_examples = get_test_examples(data_dir)
        num_actual_predict_examples = len(predict_examples)
        if FLAGS.use_tpu:
            while len(predict_examples) % FLAGS.predict_batch_size != 0:
                predict_examples.append(PaddingInputExample())

        predict_file = os.path.join(output_dir, "predict.tf_record")
        file_based_convert_examples_to_features(predict_examples, label_list,
                                                FLAGS.max_seq_length, tokenizer,
                                                predict_file)

INFO:tensorflow:Writing example 0 of 100
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] a 32 - year - old male with colon ##ic neo ##pl ##asia with liver , retro ##per ##ito ##nea ##l and media ##sti ##nal meta ##sta ##ses , with a rapid evolution over a background of ul ##cera ##tive coli ##tis . one week after diagnosis , despite presenting an advanced stage , the patient was admitted for surgical intervention with partial tumor res ##ection . post ##oper ##ative chemotherapy was initiated with different consecutive therapeutic lines ox ##ali ##pl ##atin + fox ##ali ##pl ##atin + fox ##ac ##ill ##in . the k - ras gene was even determined for possible treatment with ce ##tu ##xi ##ma ##b or pan ##it ##um ##uma ##b , being negative , and therefore ruling out this therapeutic alternative . moreover , treatment with bei ##zu ##ma ##b is not ad ##vis ##able because it drains into the abdominal cavity , producing pu ##ru ##lent material . fifteen

In [68]:
tf.logging.info("***** Running prediction*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                        len(predict_examples), num_actual_predict_examples,
                        len(predict_examples) - num_actual_predict_examples)
tf.logging.info("  Batch size = %d", FLAGS.predict_batch_size)

predict_drop_remainder = True if FLAGS.use_tpu else False
predict_input_fn = file_based_input_fn_builder(
            input_file=predict_file,
            seq_length=FLAGS.max_seq_length,
            is_training=False,
            drop_remainder=predict_drop_remainder)

INFO:tensorflow:***** Running prediction*****
INFO:tensorflow:  Num examples = 100 (100 actual, 0 padding)
INFO:tensorflow:  Batch size = 8


In [71]:
result1 = estimator.predict(input_fn=predict_input_fn)

INFO:tensorflow:prediction_loop marked as finished


In [70]:
outputs = [list(next(result1).values())[0] for i in range(100)]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?, 40)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768)

In [70]:
outputs[0]

array([0.9668534 , 0.02772105, 0.6758409 , 0.3482182 , 0.8649518 ,
       0.13742259, 0.59318393, 0.40129864, 0.63186234, 0.3400343 ,
       0.73278135, 0.2743978 , 0.82844096, 0.18083379, 0.50106436,
       0.5118963 , 0.7556841 , 0.23564705, 0.7041823 , 0.27380043,
       0.92050904, 0.08488429, 0.7450615 , 0.2723015 , 0.8092759 ,
       0.19038063, 0.80323756, 0.20033771, 0.6630727 , 0.35012436,
       0.9523307 , 0.03944254, 0.14513579, 0.8553724 , 0.79804224,
       0.19140103, 0.7500935 , 0.2134487 , 0.95894563, 0.03178847],
      dtype=float32)

In [74]:
import numpy as np

In [83]:
np_arr = np.asarray(outputs, np.float32)

In [92]:
B = np.where(np_arr > 0.5, 1, 0)
prediction = np.asarray(B,np.float32)
prediction[0]

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
       1., 0., 1., 0., 1., 0.], dtype=float32)

In [72]:
labels = [list(next(result1).values())[1] for i in range(100)]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?, 40)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768)

In [93]:
label1 = np.asarray(labels, np.float32)
label1[0]

array([1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1.,
       0., 1., 1., 0., 1., 0.], dtype=float32)

In [94]:
(prediction == label1).mean()

0.81875